<a href="https://colab.research.google.com/github/Tawanss/chatbot_project/blob/main/RAG_jason.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 ติดตั้ง libraries ที่จำเป็น
!pip install transformers datasets faiss-cpu torch

import json
import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset

# โหลด JSON file
with open('course_plan.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# แปลง JSON เป็น flat list ของ documents
documents = []
for year in data['coursePlan']:
    for semester in year['semesters']:
        if 'courses' in semester:
            for course in semester['courses']:
                doc = f"Year {year['year']} Semester {semester['semester']}: {course['code']} - {course['name']} ({course['nameEn']}) - Credits: {course['credits']}"
                documents.append(doc)
        elif 'plans' in semester:
            for plan in semester['plans']:
                for course in plan['courses']:
                    doc = f"Year {year['year']} Semester {semester['semester']} - {plan['name']}: {course['code']} - {course['name']} ({course['nameEn']}) - Credits: {course['credits']}"
                    documents.append(doc)

# สร้าง Dataset
dataset = Dataset.from_dict({"text": documents})

# โหลด tokenizer และ model
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# สร้าง function สำหรับ embedding
def embed_text(examples):
    inputs = tokenizer(examples["text"], padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return {"embeddings": embeddings.numpy()}

# สร้าง embeddings
dataset = dataset.map(embed_text, batched=True, batch_size=16)

# สร้าง FAISS index
embeddings = np.array(dataset["embeddings"]).astype("float32")
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Function สำหรับค้นหา
def search(query, top_k=5):
    query_vector = embed_text({"text": [query]})["embeddings"][0]
    distances, indices = index.search(np.array([query_vector]), top_k)
    results = [{"text": dataset[i]["text"], "distance": distances[0][j]} for j, i in enumerate(indices[0])]
    return results

# ตัวอย่างการใช้งาน
query = "วิชาภาษาอังกฤษ"
results = search(query)

print(f"Query: {query}")
for i, result in enumerate(results, 1):
    print(f"{i}. {result['text']} (Distance: {result['distance']:.4f})")

# Function สำหรับสร้าง prompt สำหรับ language model
def generate_prompt(query, results):
    prompt = f"คำถาม: {query}\n\nข้อมูลที่เกี่ยวข้อง:\n"
    for i, result in enumerate(results, 1):
        prompt += f"{i}. {result['text']}\n"
    prompt += "\nกรุณาตอบคำถามโดยใช้ข้อมูลที่ให้มา:"
    return prompt

# ตัวอย่างการสร้าง prompt
prompt = generate_prompt(query, results)
print("\nGenerated Prompt:")
print(prompt)

# หมายเหตุ: ในที่นี้เราไม่ได้เชื่อมต่อกับ language model จริง
# คุณสามารถใช้ prompt นี้กับ model ของคุณเอง เช่น GPT-3 หรือ other LLMs
# เพื่อสร้างคำตอบที่เหมาะสมต่อไป